# Pass Through do câmbio na inflação

Neste exercício, vamos seguir a publicação realizada pelo site Análise Macro, em seu Boletim AM, que sempre traz conteúdos de economia aplicada. Para conferir o desenvolvimento original do autor, segue o link:

https://analisemacro.com.br/economia/macroeconometria/estimando-o-repasse-externo-sobre-a-inflacao-brasileira/

Trarei aqui de maneira mais simplificado os temas, pois assim mantenho o papel do post original, não o "copiando" por completo e ainda desenvolvendo o  exercício.

A estimação do produto será realizado pelo *Generalized Method of Moments (GMM)*. As variáveis utilizadas no modelo são:

* Hiato do produto - BCB
* IC-Br USD (Var. % Trim.) - BCB
* Expectativa de 12 meses do IPCA Total (a.a.%) - BCB
* IPCA EX01 (a.a.%) - BCB

#### **1.** Carregando libs


In [2]:
%pip install python-bcb
%pip install linearmodels

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from bcb import sgs #importando o Sistema Gerenciador de Séries Temporais do BC
from bcb import Expectativas #importanto dados de expectativas 
import pandas as pd #para manipular os dados
import numpy as np #para manipular números e realizar cálculos
from functools import reduce # não sei
import seaborn as sns # biblioteca para visualização de dados estatistícos baseadas em matplotlib
import matplotlib.pyplot as plt # biblioteca para plotar gráficos
from plotnine import * # outra biblioteca de apoio para a construção dos gráficos

#### **2.** Coleta e tratamento de dados

##### **2.1.** Expectativas de inflação

In [4]:
# instancia a classe de expectativas
em = Expectativas()

# obtém o último ponto da expectativa da inflação acumulada em 12 meses
exp_ipca_raw = em.get_endpoint('ExpectativasMercadoInflacao12Meses')

# traz a expectativa de inflação cambial acumulada em 12 meses
exp_ipca = (
    exp_ipca_raw.query() # começa uma nova consulta
    .filter(exp_ipca_raw.Indicador == 'IPCA') # filtra só o IPCA
    .filter(exp_ipca_raw.baseCalculo == '0') # filtra a base de cálculo para 0 <<não entendi>>
    .filter(exp_ipca_raw.Suavizada == 'S') # filtra suavizada para S <<não entendi>>
    .collect() # coleta os dados da query
) 



In [5]:
exp_ipca

,Indicador,Data,Suavizada,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo
0,IPCA,2001-12-12,S,5.1700,5.2200,0.5600,NaN,NaN,NaN,0
1,IPCA,2001-12-13,S,5.1600,5.2200,0.5500,NaN,NaN,NaN,0
2,IPCA,2001-12-14,S,5.1300,5.1500,0.5700,NaN,NaN,NaN,0
3,IPCA,2001-12-17,S,5.1200,5.1600,0.5700,NaN,NaN,NaN,0
4,IPCA,2001-12-18,S,5.0900,5.1300,0.5800,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
5481,IPCA,2023-10-16,S,3.9841,4.0109,0.4064,2.9552,5.4359,130.0,0
5482,IPCA,2023-10-17,S,3.9808,3.9951,0.4043,2.9566,5.4353,131.0,0
5483,IPCA,2023-10-18,S,3.9741,3.9797,0.4033,2.9581,5.4346,132.0,0
5484,IPCA,2023-10-19,S,3.9692,3.9688,0.4007,2.9596,5.4340,132.0,0


In [6]:
# tratamento da IPCA-EX0
## o que vamos fazer é trimestralizar a expectativa (média)

exp_ipca_aux = (
  exp_ipca
  # cria colunas de ano e mês para o agrupamento
 .assign(Year = lambda x: pd.to_datetime(x.Data).dt.year,
         Month = lambda x: pd.to_datetime(x.Data).dt.month)
  # agrupa por ano e mês
  .groupby(by = ['Year', 'Month'])
  # calcula a média
  .agg({'Mediana' : 'mean'})
  .reset_index()
  # cria colunas para o trimestre e cria uma coluna de data trimestral
  .assign(Quarter = lambda x: pd.to_datetime(x[['Year', 'Month']].assign(Day = 1)),
          date_quarter = lambda x: pd.PeriodIndex(x['Quarter'], freq = 'Q'))
  # agrupa por trimestre
  .groupby(by = 'date_quarter')
  # calcula a média
  .agg(ipca_exp_12m = ('Mediana', 'mean'))
  .reset_index()
)

In [7]:
exp_ipca_aux

,date_quarter,ipca_exp_12m
0,2001Q4,5.030000
1,2002Q1,4.699958
2,2002Q2,4.541706
3,2002Q3,5.062570
4,2002Q4,9.692981
...,...,...
84,2022Q4,5.207272
85,2023Q1,5.502346
86,2023Q2,4.822036
87,2023Q3,4.141353


##### **2.2.** IPCA-EX0

In [8]:
# função para acumular os trimestres

def acum_quarter(x):
    """
    Essa função vai acumular a variação percentual em um trimestre

    Argumentos:
     x: lista, array ou série pandas contendo os valores de inflação.

    Retorno:
    inflação acumulada em um trimestre, em porcentagem.
    """
    # Transforma os valores em fatores de crescimento
    x_fac = 1 + (x/100)
    
    # Calcula o produto acumulado dos últimos três valores
    x_cum = np.prod(x_fac[-3:])

    # Calcula a inflação acumulada em um trimestre
    x_qr = (x_cum - 1)*100

    return x_qr

def acum_p (data, n):
    """
    Calcula a acumulação de variações percentuais em janelas móveis.

    Argumentos:
    data: array-like
        Uma sequência de dados representando variações percentuais.

    n: int
        O tamanho da janela móvel para o cálculo da acumulação.

    Retorno:
        array
        Um array contendo a acumulação das variações percentuais em janelas móveis.
    """

    resultado = (((data/100) +1 )
                .rolling(window=n)
                .apply(np.prod)
                -1) * 100
    return resultado



In [9]:
# coleta e tratamento do IPCA-EX0

dados_ipca = (
    # importa o ipca total e livres do sgs
    sgs.get({'ipca_nucleo':11427},
        start = '1998-01-01')
    .reset_index()

    #cria coluna de data trimestral
    .assign(date_quarter = lambda x: pd.PeriodIndex(x['Date'], freq = 'Q'))

    #agrupa por trimestre
    .groupby(by = 'date_quarter')

    #calcula o agregado trimestral
    .agg({
        'ipca_nucleo': lambda x: acum_quarter(x)
    })

    .reset_index()

    #calcula o produto móvel de 4 trimestres
    .assign(ipca_nucleo = lambda x: acum_p(x.ipca_nucleo, n=4))
)

In [10]:
dados_ipca

,date_quarter,ipca_nucleo
0,1998Q1,NaN
1,1998Q2,NaN
2,1998Q3,NaN
3,1998Q4,0.982808
4,1999Q1,1.395896
...,...,...
98,2022Q3,9.712147
99,2022Q4,8.401948
100,2023Q1,7.341027
101,2023Q2,5.310332


#### **2.3.** IC-Br USD

In [11]:
# função para acumular os trimestres 

def acum_ic(x):
    """
    Vamos seguir a mesma lógica vista antes, mas agora na série do IC-br
    outra função para calcular a variação acumulada em um determinado período da série do IC-br

    Argumento: 
    x: lista, array ou série pandas contendo os valores do IC-br.

    Retorno:
    A variação acumulada em um determinado período, em porcentagem.
    """
    #calcula a variação percentual em relação ao primeiro valor de x
    x_diff = np.log(x/x.iloc[0])*100

    #retorna o último valor da série x_diff
    x_acum = x_diff.iloc[-1]

    return x_acum


In [12]:
#coleta e tratamento do IC-Br USD

dados_ic = (
    # importa os dados do IC-Br do sgs
    sgs.get({'ic_br':29042},
        start= '1998-01-01')
    .reset_index()

    #cria a coluna de data trimestral
    .assign(date_quarter = lambda x: pd.PeriodIndex(x['Date'], freq = 'Q'))

    # agrupar por trimestres
    .groupby(by = 'date_quarter')

    #calcula o acumulado trimestral
    .agg({
        'ic_br': lambda x: acum_ic(x)
    })
    .reset_index()
)

In [13]:
dados_ic

,date_quarter,ic_br
0,1998Q1,-3.910579
1,1998Q2,-5.008094
2,1998Q3,-4.150212
3,1998Q4,-3.858045
4,1999Q1,-6.631748
...,...,...
98,2022Q3,0.387217
99,2022Q4,-0.853307
100,2023Q1,-4.661394
101,2023Q2,-2.126762


#### **2.4.** Hiato do produto

In [16]:
#Coleta e tratamento do Hiato do produto do BC
hiato = (
    # lê o arquivo direto da url
    pd.read_excel(
    "https://www.bcb.gov.br/content/ri/relatorioinflacao/202309/ri202309anp.xlsx",
    sheet_name = "Graf 2.2.4",
    skiprows = 8
    )
    # altera o tipo de dado da coluna de data e do hiato
    .assign(date_quarter = lambda x: pd.PeriodIndex(x['Trimestre'], freq = 'Q'),
            hiato = lambda x: x.Hiato.astype(float))
    .loc[:, ['date_quarter', 'hiato']]
  )


In [17]:
hiato

,date_quarter,hiato
0,2003Q4,-0.82
1,2004Q1,-0.39
2,2004Q2,-0.02
3,2004Q3,0.37
4,2004Q4,0.64
...,...,...
75,2022Q3,-0.57
76,2022Q4,-0.59
77,2023Q1,-0.64
78,2023Q2,-0.66
